#### Setting the environment variables

In [1]:
import os, sys, warnings, platform

osname = platform.system()
print('Running on', osname)

if (osname == 'Windows'):
    # Definitions for Windows 10 instance
    os.environ["PYSPARK_PYTHON"] = "D:/Anaconda3/python"
    os.environ["PYSPARK_DRIVER_PYTHON"]="D:/Anaconda3/python"
    os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook --no-browser"
    os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk1.8.0_251/jre"
    os.environ["SPARK_HOME"] = "D:/spark-2.4.4-bin-hadoop2.7"
    os.environ["HADOOP_HOME"] = "D:/spark-2.4.4-bin-hadoop2.7"
    os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
    sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
    sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
else:
    # Definitions for EC2 Linux instance
    os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
    os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
    os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook --no-browser"
    os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
    os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
    os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
    sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
    sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

Running on Linux


# Ecommerce Churn Assignment

The aim of the assignment is to build a model that predicts whether a person purchases an item after it has been added to the cart or not. Being a classification problem, you are expected to use your understanding of all the three models covered till now. You must select the most robust model and provide a solution that predicts the churn in the most suitable manner. 

For this assignment, you are provided the data associated with an e-commerce company for the month of October 2019. Your task is to first analyse the data, and then perform multiple steps towards the model building process.

The broad tasks are:
- Data Exploration
- Feature Engineering
- Model Selection
- Model Inference

### Data description

The dataset stores the information of a customer session on the e-commerce platform. It records the activity and the associated parameters with it.

- **event_time**: Date and time when user accesses the platform
- **event_type**: Action performed by the customer
            - View
            - Cart
            - Purchase
            - Remove from cart
- **product_id**: Unique number to identify the product in the event
- **category_id**: Unique number to identify the category of the product
- **category_code**: Stores primary and secondary categories of the product
- **brand**: Brand associated with the product
- **price**: Price of the product
- **user_id**: Unique ID for a customer
- **user_session**: Session ID for a user


### Initialising the SparkSession

The dataset provided is 5 GBs in size. Therefore, it is expected that you increase the driver memory to a greater number. You can refer to notebook 1 for the steps involved here.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

MAX_MEMORY = "14G"
spark = SparkSession.builder.appName("Notebook_3").config("spark.driver.memory", MAX_MEMORY).getOrCreate()
spark

In [3]:
# Spark session with 14 GB driver memory

spark.sparkContext.getConf().get('spark.driver.memory')

'14G'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('float_format', '{:.4f}'.format)
pd.options.display.max_colwidth = 100
%matplotlib inline
plt.rcParams['font.size'] = '14'

In [5]:
%%time
# Loading the clean data

df = spark.read.parquet('task3_transformed_df.parquet')
print('transformed_df row count =', df.count())

transformed_df row count = 1033889
CPU times: user 2.88 ms, sys: 1.48 ms, total: 4.36 ms
Wall time: 3.66 s


<hr>

## Task 3: Model Selection
3 models for classification:	
- Logistic Regression
- Decision Tree
- Random Forest

### Model 2: Decision Trees

In [6]:
# Additional steps for Decision Trees, if any

#### Feature Transformation (Code will be same; check for the columns)

In [7]:
# Check if only the required columns are present to build the model
# If not, drop the redundant columns

# This step is already completed as part of Notebook_2

In [8]:
# Categorising the attributes into its type - Continuous and Categorical

# This step is already completed as part of Notebook_2

In [9]:
# Feature transformation for categorical features

# This step is already completed as part of Notebook_2

In [10]:
# Vector assembler to combine all the features

# This step is already completed as part of Notebook_2

In [11]:
# Pipeline for the tasks

# This step is already completed as part of Notebook_2

In [12]:
# Transforming the dataframe df

# This step is already completed as part of Notebook_2

In [13]:
%%time
# Schema of the transformed df

df.printSchema()

root
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- user_session_activity_count: long (nullable = true)
 |-- product_count_for_user: long (nullable = true)
 |-- sub_category_count_for_user: long (nullable = true)
 |-- avg_expense_for_sub_category: double (nullable = true)
 |-- user_sessions_count: long (nullable = true)
 |-- day_quadrant: string (nullable = true)
 |-- is_purchased: integer (nullable = true)
 |-- day_of_week_idx: double (nullable = true)
 |-- day_of_week_enc: vector (nullable = true)
 |-- day_quadrant_idx: double (nullable = true)
 |-- day_quadrant_enc: vector (nullable = true)
 |-- category_idx: double (nullable = true)
 |-- category_enc: vector (nullable = true)
 |-- sub_category_idx: double (nullable = true)
 |-- sub_category_enc: vector (nullable = true)
 |-- brand_idx: double (nullable = true)
 |-- brand_enc:

In [14]:
%%time
# Checking the elements of the transformed df - Top 20 rows

df.show(20)

+-------+-------+-----------+------------+------------+---------------------------+----------------------+---------------------------+----------------------------+-------------------+------------+------------+---------------+---------------+----------------+----------------+------------+--------------+----------------+----------------+---------+--------------+--------------------+
|  brand|  price|day_of_week|    category|sub_category|user_session_activity_count|product_count_for_user|sub_category_count_for_user|avg_expense_for_sub_category|user_sessions_count|day_quadrant|is_purchased|day_of_week_idx|day_of_week_enc|day_quadrant_idx|day_quadrant_enc|category_idx|  category_enc|sub_category_idx|sub_category_enc|brand_idx|     brand_enc|            features|
+-------+-------+-----------+------------+------------+---------------------------+----------------------+---------------------------+----------------------------+-------------------+------------+------------+---------------+-------

In [15]:
# Storing the transformed df in S3 bucket to prevent repetition of steps again

# This step is already completed as part of Notebook_2

#### Train-test split

In [16]:
%%time
# Splitting the data into train and test (Remember you are expected to compare the model later)

df_train, df_test = df.randomSplit([0.7, 0.3], seed=42)

CPU times: user 2.3 ms, sys: 1.16 ms, total: 3.46 ms
Wall time: 25.4 ms


In [17]:
%%time
# Number of rows in train and test data

print('Train Dataset Count:', df_train.count(), '| Test Dataset Count:', df_test.count())

Train Dataset Count: 723091 | Test Dataset Count: 310798
CPU times: user 5.21 ms, sys: 2.35 ms, total: 7.56 ms
Wall time: 28.3 s


#### Model Fitting

In [18]:
%%time
# Building the model with hyperparameter tuning
# Create ParamGrid for Cross Validation

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

dt = DecisionTreeClassifier(featuresCol='features', labelCol='is_purchased', seed=42)

dtParamGrid = ParamGridBuilder() \
              .addGrid(dt.maxDepth, [10,20,30]) \
              .addGrid(dt.maxBins, [5,10,15]) \
              .build()

dtEvaluator = BinaryClassificationEvaluator(labelCol='is_purchased')

multiEvaluator = MulticlassClassificationEvaluator(labelCol='is_purchased', predictionCol='prediction')

CPU times: user 164 ms, sys: 3.58 ms, total: 167 ms
Wall time: 205 ms


In [19]:
%%time
# Run cross-validation steps

crossval = CrossValidator(estimator=dt, estimatorParamMaps=dtParamGrid, evaluator=dtEvaluator, numFolds=3)

CPU times: user 339 µs, sys: 0 ns, total: 339 µs
Wall time: 327 µs


In [20]:
%%time
# Fitting the models on transformed df

cvModel = crossval.fit(df)

CPU times: user 1.69 s, sys: 715 ms, total: 2.4 s
Wall time: 21min 8s


In [21]:
%%time
# Best model from the results of cross-validation

dtBestModel = cvModel.bestModel
dtBestModel

CPU times: user 19 µs, sys: 0 ns, total: 19 µs
Wall time: 24.1 µs


DecisionTreeClassificationModel (uid=DecisionTreeClassifier_a7a731a0d45c) of depth 30 with 374963 nodes

#### Model Analysis

Required Steps:
- Fit on test data
- Performance analysis
    - Appropriate Metric with reasoning

In [22]:
%%time
predictions = dtBestModel.transform(df_test)

CPU times: user 6.67 ms, sys: 2.12 ms, total: 8.79 ms
Wall time: 62.4 ms


In [23]:
%%time
multiEvaluator.setMetricName('accuracy')
accuracy = multiEvaluator.evaluate(predictions)
print('Test data Accuracy =', accuracy)

Test data Accuracy = 0.8785352544096166
CPU times: user 22.9 ms, sys: 10.1 ms, total: 33 ms
Wall time: 34.8 s


In [24]:
%%time
multiEvaluator.setMetricName('weightedPrecision')
precision = multiEvaluator.evaluate(predictions)
print('Test data Precision =', precision)

Test data Precision = 0.8790522005363621
CPU times: user 29.5 ms, sys: 12.1 ms, total: 41.5 ms
Wall time: 50.6 s


In [25]:
%%time
multiEvaluator.setMetricName('weightedRecall')
recall = multiEvaluator.evaluate(predictions)
print('Test data Recall =', recall)

Test data Recall = 0.8785352544096166
CPU times: user 26.1 ms, sys: 5.79 ms, total: 31.9 ms
Wall time: 33.5 s


In [26]:
F1_score = (2 * precision * recall) / (precision + recall)
print('Test data F1_score =', F1_score)

Test data F1_score = 0.8787936514502295


In [27]:
%%time
areaUnderROC = dtEvaluator.evaluate(predictions, {dtEvaluator.metricName: 'areaUnderROC'})
print('Test data ROC_AUC =', areaUnderROC)

Test data ROC_AUC = 0.7396132709059026
CPU times: user 13.2 ms, sys: 5.73 ms, total: 18.9 ms
Wall time: 17.5 s


#### Summary of the best Decision Tree model

In [28]:
# Defining a function to extract features along with the feature importance score

import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [39]:
%%time
# Printing the feature importance scores

dtFeatureImp = ExtractFeatureImp(dtBestModel.featureImportances, predictions, 'features')
dtFeatureImp.to_csv('decisionTreeFeatureImp.csv')
dtFeatureImp.head(5)

CPU times: user 7.9 ms, sys: 0 ns, total: 7.9 ms
Wall time: 10.3 ms


,idx,name,score
5,103,user_sessions_count,0.1235
2,100,product_count_for_user,0.1071
3,101,sub_category_count_for_user,0.1069
0,98,price,0.1055
1,99,user_session_activity_count,0.0913


__*Based on the DecisionTree featureImportance scores, we can see that the top 5 important features for predicting the "purchase" target variable are:*__
1. user_sessions_count
2. product_count_for_user
3. sub_category_count_for_user
4. price
5. user_session_activity_count

In [30]:
print('Tree depth of the best model =', dtBestModel.depth)

Tree depth of the best model = 30


In [31]:
print('Number of features used by the best model =', dtBestModel.numFeatures)

Number of features used by the best model = 104


In [32]:
print('Number of nodes in the best model =', dtBestModel.numNodes)

Number of nodes in the best model = 374963


In [37]:
# Collate all the model evaluation metrics
decisionTreeMetrics = [{'ModelType'    : 'DecisionTree',
                        'Accuracy'     : accuracy,
                        'Precision'    : precision,
                        'Recall'       : recall,
                        'F1_score'     : F1_score,
                        'AreaUnderROC' : areaUnderROC}]
  
# Convert the metrics to a Pandas dataframe 
decisionTreeMetrics_df = pd.DataFrame(decisionTreeMetrics)

# Save the dataframe as csv for future model comparison
decisionTreeMetrics_df.to_csv('decisionTreeMetrics.csv')

decisionTreeMetrics_df

,ModelType,Accuracy,Precision,Recall,F1_score,AreaUnderROC
0,DecisionTree,0.8785,0.8791,0.8785,0.8788,0.7396


### Evaluation metrics for the best DecisionTree Model :
1. Accuracy = 0.8785352544096166<br>
2. Precision = 0.8790522005363621<br>
3. Recall = 0.8785352544096166<br>
4. F1_score = 0.8787936514502295<br>
5. AreaUnderROC = 0.7396132709059026<br>